# Analyse french stocks

In this tutorial, we will use the basic technics to analyse all french stock and try to find out which stock is worth for investment.

The objectives:
1. Find the top 20 stock with the best dividendYield of the last 5 years
2. Find the top 20 stock which are under evaluated ()
3. Find the top 20 stock which lost values in percentage.

## 1. Get all french stock symbols (ticker)

We can download the list of the stock symbols from this [site](https://www.abcbourse.com/download/libelles)



In [5]:
import pandas as pd
import yfinance as yf
import datetime

In [2]:
filePath= "../data/fr_stocks.csv"

stockSymbolDf = pd.read_csv(filePath, sep=";")
stockSymbolDf.head(5)

,ISIN,nom,ticker
0,FR0000031122,Air France - KLM,AF
1,FR0000031577,Virbac,VIRP
2,FR0000031775,Vicat,VCT
3,FR0000032278,Latecoere,LAT
4,FR0000032526,Guerbet,GBT


In [11]:
# we have 216 stock in our list
stockSymbolDf.shape

(216, 3)

We can notice that the ticker is not valid for yf, We need to add `.PA` to complete. For example, we need to convert `AF` to `AF.PA`

In [8]:
bad = yf.Ticker("AF")
print(bad.info)

{'exchange': 'YHD', 'quoteType': 'MUTUALFUND', 'symbol': 'AF', 'underlyingSymbol': 'AF', 'shortName': '411094', 'timeZoneFullName': 'America/New_York', 'timeZoneShortName': 'EDT', 'uuid': '06fa567b-72d7-3b7e-bea7-46973fbbd1d5', 'gmtOffSetMilliseconds': -14400000, 'maxAge': 86400, 'trailingPegRatio': None}


In [9]:
good=yf.Ticker("AF.PA")
print(good.info)

{'address1': '7 rue du Cirque', 'city': 'Paris', 'zip': '75008', 'country': 'France', 'phone': '33 1 43 17 20 20', 'website': 'https://www.airfranceklm.com', 'industry': 'Airlines', 'industryDisp': 'Airlines', 'sector': 'Industrials', 'sectorDisp': 'Industrials', 'longBusinessSummary': 'Air France-KLM SA, together with its subsidiaries, provides passenger and cargo transportation services and aeronautical maintenance in Metropolitan France, Benelux, rest of Europe, and internationally. The company operates through three segments: Airframe maintenance, Engine Maintenance, and Component Support. It offers airframe maintenance, electronic, mechanical, pneumatic, hydraulic, and other services. The company was founded in 1919 and is headquartered in Paris, France.', 'fullTimeEmployees': 74580, 'companyOfficers': [{'maxAge': 1, 'name': 'Mr. Benjamin M. Smith', 'age': 51, 'title': 'CEO & Director', 'yearBorn': 1971, 'fiscalYear': 2022, 'totalPay': 2235149, 'exercisedValue': 0, 'unexercisedVal

In [10]:
# convert
postfix=".PA"
stockSymbolDf['ticker'] = stockSymbolDf['ticker']+postfix
stockSymbolDf.head(5)

,ISIN,nom,ticker
0,FR0000031122,Air France - KLM,AF.PA
1,FR0000031577,Virbac,VIRP.PA
2,FR0000031775,Vicat,VCT.PA
3,FR0000032278,Latecoere,LAT.PA
4,FR0000032526,Guerbet,GBT.PA


## 2. Enrich the stock data frame

We need more information to judge a stock.

### 2.1 Which sector the enterprise works on ?

In [13]:
tickerList = stockSymbolDf['ticker'].tolist()

tickerDictList = []
for ticker in tickerList:
    tickerInfo = yf.Ticker(ticker).info
    tickerDictList.append(tickerInfo)


tickerDf = pd.DataFrame(tickerDictList)
tickerDf.head(5)

,address1,city,zip,country,phone,website,industry,industryDisp,sector,sectorDisp,...,financialCurrency,trailingPegRatio,address2,dividendRate,dividendYield,priceToBook,debtToEquity,returnOnEquity,fiveYearAvgDividendYield,fax
0,7 rue du Cirque,Paris,75008,France,33 1 43 17 20 20,https://www.airfranceklm.com,Airlines,Airlines,Industrials,Industrials,...,EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13e rue LID - BP 27,Carros,06511,France,33 4 92 08 71 00,https://corporate.virbac.com,Drug Manufacturers—General,Drug Manufacturers—General,Healthcare,Healthcare,...,EUR,NaN,Cedex,1.32,0.0048,2.635232,11.545,0.15512,NaN,NaN
2,4 Rue Aristide Berges,L'Isle-d'Abeau,38080,France,33 4 74 27 59 00,https://www.vicat.fr,Building Materials,Building Materials,Basic Materials,Basic Materials,...,EUR,NaN,Les Trois Vallons,1.65,0.0518,0.546824,73.292,0.06851,4.69,NaN
3,"135, rue de Periole",Toulouse,31079,France,33 5 61 58 77 00,https://www.latecoere.aero,Aerospace & Defense,Aerospace & Defense,Industrials,Industrials,...,EUR,NaN,BP 25211 Cedex 5,NaN,NaN,2.962500,1839.217,-1.00543,NaN,NaN
4,"15, rue des Vanesses",Villepinte,93420,France,33 1 45 91 50 00,https://www.guerbet.com,Medical Devices,Medical Devices,Healthcare,Healthcare,...,EUR,NaN,Zone Paris Nord II,0.50,0.0230,0.708182,82.209,-0.10480,2.60,33 1 45 91 51 99


In [14]:
# set the symbol colum as the index column.
# The inplace=True parameter updates the DataFrame in place, modifying it without needing to assign the result back to the variable.
tickerDf.set_index("symbol",inplace=True)
tickerDf.head(5)

,address1,city,zip,country,phone,website,industry,industryDisp,sector,sectorDisp,...,financialCurrency,trailingPegRatio,address2,dividendRate,dividendYield,priceToBook,debtToEquity,returnOnEquity,fiveYearAvgDividendYield,fax
symbol,,,,,,,,,,,,,,,,,,,,,
AF.PA,7 rue du Cirque,Paris,75008,France,33 1 43 17 20 20,https://www.airfranceklm.com,Airlines,Airlines,Industrials,Industrials,...,EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VIRP.PA,13e rue LID - BP 27,Carros,06511,France,33 4 92 08 71 00,https://corporate.virbac.com,Drug Manufacturers—General,Drug Manufacturers—General,Healthcare,Healthcare,...,EUR,NaN,Cedex,1.32,0.0048,2.635232,11.545,0.15512,NaN,NaN
VCT.PA,4 Rue Aristide Berges,L'Isle-d'Abeau,38080,France,33 4 74 27 59 00,https://www.vicat.fr,Building Materials,Building Materials,Basic Materials,Basic Materials,...,EUR,NaN,Les Trois Vallons,1.65,0.0518,0.546824,73.292,0.06851,4.69,NaN
LAT.PA,"135, rue de Periole",Toulouse,31079,France,33 5 61 58 77 00,https://www.latecoere.aero,Aerospace & Defense,Aerospace & Defense,Industrials,Industrials,...,EUR,NaN,BP 25211 Cedex 5,NaN,NaN,2.962500,1839.217,-1.00543,NaN,NaN
GBT.PA,"15, rue des Vanesses",Villepinte,93420,France,33 1 45 91 50 00,https://www.guerbet.com,Medical Devices,Medical Devices,Healthcare,Healthcare,...,EUR,NaN,Zone Paris Nord II,0.50,0.0230,0.708182,82.209,-0.10480,2.60,33 1 45 91 51 99


In [16]:
outPath= "../data/fr_stock_info.csv"

In [17]:
tickerDf.to_csv(outPath,index=True)